# Handling VERITAS spectra - Structuring the plot environment

I want here to separate the data sources from the plot as best as possible.
Ideally, plot should not keep any information from data sources but a reference to it.

In [1]:
%ls

handling_mkn421-Adding interaction to plots.ipynb
handling_mkn421.ipynb
Mkn421_VERITAS_2008_highA.csv
Mkn421_VERITAS_2008_highB.csv
Mkn421_VERITAS_2008_highC.csv
Mkn421_VERITAS_2008_low.csv
Mkn421_VERITAS_2008_mid.csv
Mkn421_VERITAS_2008_veryhigh.csv
Mkn421_VERITAS_2008_verylow.csv
setd.html


In [2]:
def mjd_header2table(table):
    def header2table(table, header_keyword, datatype=float, unitname='day'):
        from astropy.table import Column
        hfield = None
        colname = None
        if isinstance(header_keyword,(str,unicode)):
            hfield = table.meta[header_keyword]
            colname = header_keyword
        else:
            assert isinstance(header_keyword,(list,tuple))
            colname = '_'.join(header_keyword)
            for keyword in header_keyword:
                if hfield is None:
                    hfield = table.meta
                hfield = hfield.get(keyword)
        col = Column(data = [hfield]*len(table), 
                     name = colname, dtype=datatype, 
                     unit = unitname)
        table.add_column(col)

    header2table(table, ('mjd','start'))
    header2table(table, ('mjd','end'))
    table['mjd'] = table['mjd_start']
    table['mjd_delta'] = table['mjd_end'] - table['mjd_start']
    del table['mjd_start'],table['mjd_end']


## On plotting, add some interaction

I want to add some interaction on those plots.
For instance, I want to share properties (like shading/selecting) of those data points.
To accomplish that, I have to use an structure called `data source`; plotting facilities will share the same `DataSource`, which works as a hub of information for all the linked facilities.

This means that the way plots are being created needs to be changed, also to be changed is the way data is pushed to the plots.
For each *spectra* (i.e, table), a new DataSource needs to be created and both plots (SED and TED) are updated.

In [7]:

from collections import namedtuple
ColumnsSED = namedtuple('ColumnsSED', ['flux', 'flux_errp', 'flux_errn',
                                       'energy', 'epoch', 'epoch_errp'])

import astropy
class DatasetSED(object):
    _tablecols = ColumnsSED( 'dnde', 'dnde_errp', 'dnde_errn',
                             'e_ref', 'mjd', 'mjd_delta' )

    def __init__(self,table):
        from astropy.table import Table
        assert isinstance(table,Table)
        
    def _datasource_from_table(self,table):
        '''
        Returns a ~bokeh.models.ColumnDataSource from ~astropy.table.Table
        
        'table' is expected to provide columns '%s'.
        '''%(self._tablecols)
        # Guarantee we have all columns for [ST]ED plots
        assert all( c in self._columns for c in table.colnames )
        
        from astropy.time import Time
        from bokeh.models import ColumnDataSource

        self.set_units(table)
        
        data = {
            self.energy    : table[self.energy]
            }
        
        data.update({
            self.flux      : table[self.flux],
            self.flux_errp : table[self.flux] + table[self.flux_errp],
            self.flux_errn : table[self.flux] - table[self.flux_errn]
            })
        
        data.update({
            self.epoch      : Time(table[self.epoch],format='mjd').datetime,
            self.epoch_errp : Time(table[self.epoch] + table[self.epoch_errp], format='mjd').datetime,
            })

        ds = ColumnDataSource(data=data)
        ds_index = len(self.datasources)
        ds_label = table.meta.get('object') +':'+ str(table.meta.get('mjd').get('start'))
        i = None
        while self.datasources.has_key(ds_label):
            if not i:
                i = 2
                ds_label += '_{}'.format(i)
            else:
                i += 1
                ds_label = ds_label[-1] + str(i)
        self.datasources[ds_label] = { 'data':ds, 'index':ds_index, 'glyphs':[] }
        return ds_label

    
class PlotFlux(object):
    flux = 'dnde'
    flux_errp = '{}_errp'.format(flux)
    flux_errn = '{}_errn'.format(flux)
    epoch = 'mjd'
    epoch_errp = '{}_delta'.format(epoch)
    energy = 'e_ref'
    _columns = [ flux, flux_errp, flux_errn,
               epoch, epoch_errp,
               energy ]
    
    NUMAX = 16 # maximum number of datasets
    
    def __init__(self):
        super(PlotFluxInt,self).__init__()
        
        self.set_palette()
        self.set_tools()
        
        from collections import OrderedDict
        self.datasources = OrderedDict()
        self.figures = {}
        self.checkbox = None
        
    def set_palette(self):
        from bokeh.palettes import viridis,magma
        colors = viridis(self.NUMAX/2)
        colors.extend(magma(self.NUMAX/2))
        from random import shuffle,seed
        seed(1234567)
        shuffle(colors)
        self.palette = colors[:]
        
    def set_tools(self):
        _tools = [  'pan',
                    'box_zoom',
                    'resize',
                    'reset',
                    'box_select',
                    'lasso_select',
                    'help']
        self.tools = _tools

    def add_dataset(self,table):
#         assert all( c in self._columns for c in table.colnames )
#         from astropy.time import Time
#         from bokeh.models import ColumnDataSource

#         self.set_units(table)
        
#         data = {
#             self.energy    : table[self.energy]
#             }
        
#         data.update({
#             self.flux      : table[self.flux],
#             self.flux_errp : table[self.flux] + table[self.flux_errp],
#             self.flux_errn : table[self.flux] - table[self.flux_errn]
#             })
        
#         data.update({
#             self.epoch      : Time(table[self.epoch],format='mjd').datetime,
#             self.epoch_errp : Time(table[self.epoch] + table[self.epoch_errp], format='mjd').datetime,
#             })

#         ds = ColumnDataSource(data=data)
#         ds_index = len(self.datasources)
#         ds_label = table.meta.get('object') +':'+ str(table.meta.get('mjd').get('start'))
#         i = None
#         while self.datasources.has_key(ds_label):
#             if not i:
#                 i = 2
#                 ds_label += '_{}'.format(i)
#             else:
#                 i += 1
#                 ds_label = ds_label[-1] + str(i)
#         self.datasources[ds_label] = { 'data':ds, 'index':ds_index, 'glyphs':[] }
#         return ds_label
    
#     def create_checkboxgroup(self):
#         from bokeh.models import CheckboxGroup
#         labels = self.datasources.keys()
#         checkbox = CheckboxGroup(labels=labels,
#                                  active=range(len(labels)),
#                                  inline=True)

#         def define_callback(ds,checkbox):
#             ds_glyphs = { lbl:ds[lbl].get('glyphs') for lbl in ds.keys()}
#             def update(attr,old,new):
#                 for i,label in enumerate(checkbox.labels):
#                     for g in ds_glyphs[label]:
#                         g.visible = False#i in checkbox.active
#             return update
#         foo = define_callback(self.datasources,checkbox)
#         checkbox.on_change('active',foo)
#         return checkbox
        
    def plot(self,ds_label):
        if not self.figures:
            self.create_figures()
        self.sed(ds_label,self.figures['sed'])
        self.ted(ds_label,self.figures['ted'])
        
#         _checkbox = self.create_checkboxgroup()

#         from bokeh.layouts import gridplot
#         grid = gridplot( [[self.figures['sed']],
#                           [self.figures['ted']],
#                           [_checkbox]] )
#         return grid

    def get_figures(self):
        return self.figures
    
    def set_units(self,table):
        self.flux_unit = table[self.flux].unit.to_string()
        self.epoch_unit = table[self.epoch].unit.to_string()
        self.energy_unit = table[self.energy].unit.to_string()

    def get_color(self,ds_label):
        ds_index = self.datasources[ds_label].get('index')
        return self.palette[ds_index]

    def create_figures(self):
        from bokeh.plotting import figure
        
        x_label = self.energy
        y_label = self.flux
        sed = figure(title='Spectral Energy Distribution',
                     x_axis_label=x_label,
                     y_axis_label=y_label,
                     y_axis_type = 'log',
                     height = 300,
                     width = 600,
                     tools=self.tools)
        self.figures['sed'] = sed

        x_label = self.epoch
        y_label = self.flux
        from bokeh.models import DatetimeTickFormatter
        ted = figure(title='Time Emission Distribution',
                     x_axis_label=x_label,
                     y_axis_label=y_label,
                     y_axis_type = 'log',
                     x_axis_type = 'datetime',
                     y_range=sed.y_range,
                     height = int(sed.plot_height),
                     width = int(sed.plot_width),
                     tools=self.tools)
        ted.xaxis.formatter = DatetimeTickFormatter(
                                                    hours=["%d %B %Y"],
                                                    days=["%d %B %Y"],
                                                    months=["%d %B %Y"],
                                                    years=["%d %B %Y"],
                                                    )
        self.figures['ted'] = ted
        
    def sed(self, ds_label, fig):
        assert fig != None
        
        ds = self.datasources[ds_label].get('data')
        color = self.get_color(ds_label)
        
        glyphs = []
#         fig.multi_line(xs = zip(xs,xs),
#                        ys = zip(ys_errn,ys_errp),
#                        color=color)

        glyph_c = fig.circle(self.energy,
                               self.flux,
                               source = ds,
                               color=color)
        glyphs.append(glyph_c)
        self.datasources[ds_label].get('glyphs').extend(glyphs)
        
    def ted(self, ds_label, fig):
        assert fig != None
        
        ds = self.datasources[ds_label].get('data')
        color = self.get_color(ds_label)

        glyphs = []
#         fig.multi_line(xs = zip(xs,xs_errp),
#                        ys = zip(ys,ys),
#                        color=color,
#                        alpha=0.25
#                       )

        glyph_c = fig.circle(self.epoch,
                               self.flux,
                               source = ds,
                               color=color)
        glyphs.append(glyph_c)
        self.datasources[ds_label].get('glyphs').extend(glyphs)


Loading BokehJS ...

In [8]:
from glob import glob
table_files = glob('Mkn*.csv')

p = PlotFluxInt()
fig = None
from astropy.table import Table
for fname in table_files:
    tt = Table.read(fname, format='ascii.ecsv')
    mjd_header2table(tt)
    ds = p.add_dataset(tt)
    p.plot(ds)

In [9]:
ds = p.datasources
ds_glyphs = { lbl:ds[lbl].get('glyphs') for lbl in ds.keys()}

In [18]:
def define_update(gryphs):
    def foo(change):
        if change['type'] == 'change' and change['name'] == 'value':
            for gryph in gryphs:
                gryph.visible = change['new']
            push_notebook()
    return foo

from ipywidgets import Checkbox,HBox,VBox
wdgts = []
for label,glyph in ds_glyphs.iteritems():
    cbox = Checkbox(value=True, description=label)
    cbox.observe( define_update(glyph) )
    wdgts.append( cbox )

num_cols = 4
num_rows = sum( bool(n) for n in divmod(len(wdgts),num_cols) )
vert = []
for v in range(num_rows):
    horz = []
    for h in range(num_cols):
        if len(wdgts):
            horz.append(wdgts.pop(0))
    vert.append( VBox(horz) )
assert len(wdgts)==0
HBox(vert)

In [19]:
figs = p.get_figures()
from bokeh.layouts import column
plot_handle = show(column(figs['sed'],figs['ted']), notebook_handle=True)

In [ ]:
# def update(**kwargs):
#     for i,label in enumerate(kwargs.keys()):
#         for g in ds_glyphs[label]:
#             g.visible = kwargs[label]
#     push_notebook()
# from ipywidgets import interact
# checkbts = { l:True for l in ds_glyphs.keys() }
# interact(update, **checkbts)